In [2]:
import numpy as np

In [4]:
n_vertices = 3
h = 20              # mm
l_edge = 10         # mm
theta = 90          # degrees

In [5]:
t_bed = 60          # degrees C
t_ext = 210         # degrees C

h_layer = 0.2       # mm
feed = 40           # mm/s

v_fan = 100         # %

In [15]:
text = 

In [16]:
text[1][0] = 'A'
print(text)

[[0], ['A']]
